In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

df_all = pd.read_csv("yearly_full_release_long_format.csv")


In [2]:

df_worldmap = df_all[
    (df_all['Area type'] == 'Country or economy') &
    (df_all['Category'] == 'Electricity generation') &
    (df_all['Subcategory'] == 'Fuel') &
    (df_all['Unit'] == 'TWh')].copy()
   

In [3]:
fuel_to_group = {
    "Wind": "Emission Free Sources",
    "Hydro": "Emission Free Sources",
    "Solar": "Emission Free Sources",
    "Other renewables": "Emission Free Sources",
    "Bioenergy": "Emission Free Sources",
    "Nuclear": "Nuclear",
    "Coal": "Fossils",
    "Other Fossils": "Fossils",
    "Gas": "Fossils",
}

In [4]:
df_worldmap["Group"] = df_worldmap["Variable"].map(fuel_to_group)

df_worldmap = df_worldmap.dropna(subset=["Group"])

df_grouped = (
    df_worldmap
    .groupby(["Year", "ISO 3 code", "Group"], as_index=False)["Value"]
    .sum()
)

df_grouped.head(10)

,Year,ISO 3 code,Group,Value
0,2000,ABW,Emission Free Sources,0.00
1,2000,ABW,Fossils,0.00
2,2000,ABW,Nuclear,0.00
3,2000,AFG,Emission Free Sources,0.31
4,2000,AFG,Fossils,0.00
5,2000,AFG,Nuclear,0.00
6,2000,AGO,Emission Free Sources,0.91
7,2000,AGO,Fossils,0.00
8,2000,AGO,Nuclear,0.00
9,2000,ALB,Emission Free Sources,4.55


In [5]:
def normalize_0_100(group: pd.DataFrame) -> pd.DataFrame:
    v = group["Value"]
    v_min = v.min()
    v_max = v.max()

    if v_max == v_min:
        group["Value_norm"] = 0.0
    else:
        group["Value_norm"] = (v - v_min) / (v_max - v_min) * 100

    return group

In [6]:
df_norm = df_grouped.copy()

df_norm["Value_norm"] = (
    df_norm
    .groupby(["Year", "Group"])["Value"]
    .transform(
        lambda x: (x - x.min()) / (x.max() - x.min()) * 100
        if x.max() != x.min() else 0.0
    )
)

In [7]:
def plot_worldmap_group(df_norm, group_to_plot):
    
    colorscales = {
        "Emission Free Sources": blues_white,
        "Nuclear": greens_white,
        "Fossils": reds_white,
    }
    
    if group_to_plot not in colorscales:
        raise ValueError(f"Gruppo non riconosciuto: {group_to_plot}")
    
    colorscale = colorscales[group_to_plot]
    
    
    df_sel = df_norm[df_norm["Group"] == group_to_plot].copy()
    
   
    df_sel["Year"] = df_sel["Year"].astype(str)
    
    fig = px.choropleth(
        df_sel,
        locations="ISO 3 code",      
        locationmode="ISO-3",
        color="Value_norm",          
        hover_name="ISO 3 code",     
        animation_frame="Year",      
        color_continuous_scale=colorscale,
        range_color=(0, 100),
        title=f"Reliance on {group_to_plot} (index 0–100)"
    )
    
    fig.update_layout(
        coloraxis_colorbar=dict(title="Index (0–100)")
    )
    
    fig.show()

In [8]:

blues_white = [
    [0.0, "#ffffff"],
    [1.0, "#08306b"]
]


greens_white = [
    [0.0, "#ffffff"],
    [1.0, "#00441b"]
]


reds_white = [
    [0.0, "#ffffff"],
    [1.0, "#67000d"]
]

In [9]:
plot_worldmap_group(df_norm, "Emission Free Sources")


In [10]:

plot_worldmap_group(df_norm, "Nuclear")


In [11]:

plot_worldmap_group(df_norm, "Fossils")

In [12]:
df_total = (
    df_grouped
    .groupby(["Year", "ISO 3 code"], as_index=False)["Value"]
    .sum()
    .rename(columns={"Value": "Total_TWh"})
)
df_total.head(10)

,Year,ISO 3 code,Total_TWh
0,2000,ABW,0.00
1,2000,AFG,0.31
2,2000,AGO,0.91
3,2000,ALB,4.55
4,2000,ARE,39.94
5,2000,ARG,85.84
6,2000,ARM,5.78
7,2000,ASM,0.00
8,2000,ATG,0.00
9,2000,AUS,214.56


In [13]:
df_share = df_grouped.merge(
    df_total,
    on=["Year", "ISO 3 code"],
    how="left"
)

df_share["Share_pct"] = (
    df_share["Value"] / df_share["Total_TWh"] * 100
)

df_share.head(10)

,Year,ISO 3 code,Group,Value,Total_TWh,Share_pct
0,2000,ABW,Emission Free Sources,0.00,0.00,NaN
1,2000,ABW,Fossils,0.00,0.00,NaN
2,2000,ABW,Nuclear,0.00,0.00,NaN
3,2000,AFG,Emission Free Sources,0.31,0.31,100.0
4,2000,AFG,Fossils,0.00,0.31,0.0
5,2000,AFG,Nuclear,0.00,0.31,0.0
6,2000,AGO,Emission Free Sources,0.91,0.91,100.0
7,2000,AGO,Fossils,0.00,0.91,0.0
8,2000,AGO,Nuclear,0.00,0.91,0.0
9,2000,ALB,Emission Free Sources,4.55,4.55,100.0


In [14]:
df_norm = df_share.copy()

df_norm["Value_norm"] = (
    df_norm
    .groupby(["Year", "Group"])["Share_pct"]
    .transform(
        lambda x: (x - x.min()) / (x.max() - x.min()) * 100
        if x.max() != x.min() else 0.0
    )
)

In [41]:
def plot_worldmap_group_share(df_norm, group_to_plot):
    
    colorscales = {
        "Emission Free Sources": blues_white,
        "Nuclear": greens_white,
        "Fossils": reds_white,
    }
    
    if group_to_plot not in colorscales:
        raise ValueError(f"Gruppo non riconosciuto: {group_to_plot}")
    
    colorscale = colorscales[group_to_plot]
    
    df_sel = df_norm[df_norm["Group"] == group_to_plot].copy()
    
    
    df_sel["Year"] = df_sel["Year"].astype(str)
    
    fig = px.choropleth(
        df_sel,
        
        locations="ISO 3 code",      
        locationmode="ISO-3",
        color="Value_norm",          
        hover_name="ISO 3 code",     
        animation_frame="Year",      
        color_continuous_scale=colorscale,
        range_color=(0, 100),
        title=f"Reliance on {group_to_plot}, more faded = less reliant"
    )
    
    fig.update_layout(
        width = 1100,
        height = 650,
        coloraxis_colorbar=dict(title="0 = least reliant, 100 = most reliant")
    )
    
    fig.show()
    return fig

In [42]:
fig = plot_worldmap_group_share(df_norm, "Nuclear")

In [43]:
fig.write_html("worldmap_nuclear.html", include_plotlyjs="cdn")

In [44]:
fig = plot_worldmap_group_share(df_norm, "Fossils")

In [45]:
fig.write_html("worldmap_fossils.html", include_plotlyjs="cdn")

In [46]:
fig = plot_worldmap_group_share(df_norm, "Emission Free Sources")

In [47]:
fig.write_html("worldmap_renewables.html", include_plotlyjs="cdn")
